In [28]:
import numpy as np
import pandas as pd

In [2]:
! pip install transformers datasets
! pip install adapter-transformers
! pip install sklearn

In [3]:
import json

In [4]:
df = pd.read_json(r'data\rct_sample_train.jsonl', lines=True)
df = df.drop('metadata', axis=1)

In [5]:
df.head()

,label,text
0,CONCLUSIONS,Use of the mobile application was greater than...
1,RESULTS,Between-group effect sizes were 0.78 ( P < .00...
2,CONCLUSIONS,Future investigations on the efficacy and safe...
3,RESULTS,GLP-1 and BNP were infused in incremental dose...
4,RESULTS,Women compared with men had higher ischemic st...


In [6]:
labels_df = df.label.unique()
print(labels_df)

['CONCLUSIONS' 'RESULTS' 'OBJECTIVE' 'METHODS' 'BACKGROUND']


In [7]:
from datasets import load_dataset
dataset = load_dataset("json", data_files={"train": "data\\rct_sample_train.jsonl", "test": "data\\rct_sample_test.jsonl"})
dataset["train"].features

Using custom data configuration default-5834773ef589f384
Reusing dataset json (C:\Users\Sudeshna_Dash\.cache\huggingface\datasets\json\default-5834773ef589f384\0.0.0\a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253)


  0%|          | 0/2 [00:00<?, ?it/s]

{'label': Value(dtype='string', id=None),
 'text': Value(dtype='string', id=None),
 'metadata': Value(dtype='string', id=None)}

In [8]:
dataset["test"]["label"]

['BACKGROUND',
 'RESULTS',
 'RESULTS',
 'RESULTS',
 'RESULTS',
 'RESULTS',
 'RESULTS',
 'CONCLUSIONS',
 'CONCLUSIONS',
 'BACKGROUND',
 'BACKGROUND',
 'METHODS',
 'METHODS',
 'METHODS',
 'RESULTS',
 'RESULTS',
 'RESULTS',
 'RESULTS',
 'RESULTS',
 'RESULTS',
 'RESULTS',
 'CONCLUSIONS',
 'BACKGROUND',
 'METHODS',
 'METHODS',
 'METHODS',
 'RESULTS',
 'RESULTS',
 'RESULTS',
 'RESULTS',
 'CONCLUSIONS',
 'BACKGROUND',
 'BACKGROUND',
 'BACKGROUND',
 'METHODS',
 'METHODS',
 'METHODS',
 'METHODS',
 'RESULTS',
 'RESULTS',
 'RESULTS',
 'RESULTS',
 'CONCLUSIONS',
 'CONCLUSIONS',
 'OBJECTIVE',
 'OBJECTIVE',
 'OBJECTIVE',
 'OBJECTIVE',
 'METHODS',
 'METHODS',
 'METHODS',
 'METHODS',
 'METHODS',
 'RESULTS',
 'RESULTS',
 'RESULTS',
 'CONCLUSIONS',
 'CONCLUSIONS',
 'OBJECTIVE',
 'METHODS',
 'METHODS',
 'METHODS',
 'METHODS',
 'METHODS',
 'RESULTS',
 'RESULTS',
 'RESULTS',
 'CONCLUSIONS',
 'CONCLUSIONS',
 'OBJECTIVE',
 'METHODS',
 'METHODS',
 'METHODS',
 'METHODS',
 'METHODS',
 'RESULTS',
 'RESULTS',
 'R

In [12]:
dataset['train'] = dataset['train'].class_encode_column("label")

ValueError: Class encoding is only supported for Value column, and column label is ClassLabel.

In [13]:
dataset['test'] = dataset['test'].class_encode_column("label")

ValueError: Class encoding is only supported for Value column, and column label is ClassLabel.

In [14]:
dataset['train'].features

{'label': ClassLabel(num_classes=5, names=['BACKGROUND', 'CONCLUSIONS', 'METHODS', 'OBJECTIVE', 'RESULTS'], id=None),
 'text': Value(dtype='string', id=None),
 'metadata': Value(dtype='string', id=None)}

In [15]:
dataset = dataset.remove_columns(["metadata"])
dataset["train"][0]

{'label': 1,
 'text': 'Use of the mobile application was greater than in a previous trial and was associated with greater sun protection , especially among women .'}

# Roberta Training

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

In [ ]:
# from datasets import ClassLabel
# labels = ClassLabel(names=labels_df)
# print(labels)

In [ ]:
def preprocess_function(batch):
    tokenized = tokenizer(batch["text"], padding = "max_length", truncation=True)
#     tokenized['label'] = labels.str2int(batch['label'])
    return tokenized

In [ ]:
tokenized_citation = dataset.map(preprocess_function, batched=True)

In [ ]:
tokenized_citation["train"][0]

In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=labels_df.size)

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [21]:
from datasets import load_metric
metric = load_metric("accuracy")

ImportError: To be able to use accuracy, you need to install the following dependency: sklearn.
Please install it using 'pip install sklearn' for instance'

In [22]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [23]:
from transformers import TrainingArguments, Trainer 
training_args = TrainingArguments(output_dir="./results", learning_rate=2e-5, per_device_train_batch_size=16, per_device_eval_batch_size=16, num_train_epochs=5, weight_decay=0.01,evaluation_strategy="epoch")

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_citation["train"],
    eval_dataset=tokenized_citation["test"],
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

#  Adapter based Training


In [35]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def encode_batch(batch):
  return tokenizer(batch["text"], max_length=80, truncation=True, padding="max_length")

dataset = dataset.map(encode_batch, batched=True)


loading file https://huggingface.co/roberta-base/resolve/main/vocab.json from cache at C:\Users\Sudeshna_Dash/.cache\huggingface\transformers\d3ccdbfeb9aaa747ef20432d4976c32ee3fa69663b379deb253ccfce2bb1fdc5.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab
loading file https://huggingface.co/roberta-base/resolve/main/merges.txt from cache at C:\Users\Sudeshna_Dash/.cache\huggingface\transformers\cafdecc90fcab17011e12ac813dd574b4b3fea39da6dd817813efa010262ff3f.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
loading file https://huggingface.co/roberta-base/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/roberta-base/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/roberta-base/resolve/main/tokenizer_config.json from cache at None
loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at C:\Users\Sudeshna_Dash/.cache\huggingface\trans

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/31 [00:00<?, ?ba/s]

AttributeError: 'Dataset' object has no attribute 'rename_column_'

In [45]:
# The transformers model expects the target class column to be named "labels"
dataset = dataset.rename_column("label", "labels")
# Transform to pytorch tensors and only output the required columns
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

In [46]:
from transformers import RobertaConfig, RobertaAdapterModel

config = RobertaConfig.from_pretrained(
    "roberta-base",
    num_labels=labels_df.size,
)
model = RobertaAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)

loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at C:\Users\Sudeshna_Dash/.cache\huggingface\transformers\733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_i

In [47]:
# Add a new adapter
model.add_adapter("roberta_base_rct")
model.add_classification_head(
    "roberta_base_rct",
    num_labels=labels_df.size,
    id2label={ 0: labels_df[0], 1: labels_df[1], 2: labels_df[2], 3: labels_df[3], 4: labels_df[4]}
  )
# Activate the adapter
model.train_adapter("roberta_base_rct")

Adding adapter 'roberta_base_rct'.
Adding head 'roberta_base_rct' with config {'head_type': 'classification', 'num_labels': 5, 'layers': 2, 'activation_function': 'tanh', 'label2id': {'CONCLUSIONS': 0, 'RESULTS': 1, 'OBJECTIVE': 2, 'METHODS': 3, 'BACKGROUND': 4}, 'use_pooler': False, 'bias': True}.


In [48]:
from transformers import TrainingArguments, AdapterTrainer, EvalPrediction

training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=6,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

def compute_accuracy(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis=1)
  return {"acc": (preds == p.label_ids).mean()}

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    compute_metrics=compute_accuracy,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [50]:
trainer.train()

***** Running training *****
  Num examples = 500
  Num Epochs = 6
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 96


Step,Training Loss


KeyboardInterrupt: 

In [ ]:
trainer.evaluate()

In [ ]:
from transformers import TextClassificationPipeline

classifier = TextClassificationPipeline(model=model, tokenizer=tokenizer, device=training_args.device.index)

classifier("These results are great for future purpose")

In [ ]:
model.save_adapter("./final_adapter", "rotten_tomatoes")

!ls -lh final_adapter